In [152]:
import pandas as pd
import numpy as np

Read in Drug Raw Data inside data folder

In [153]:
drg = pd.read_csv('../data/RawData_Drug_246_12_15_2017.csv',encoding='latin1')
drg

,Drug A,Age of patient,Patient gender,Height,Tumor stage,Date enrolled,Complications?
0,1,25,Male,61Ó,1,1/15/99,no
1,2,?,female,5Õ8Ó,2,2/1/99,y
2,3,40,male,120cm,IV,2/13/99,No
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Drug B,NaN,NaN,NaN,NaN,NaN,NaN
5,1,unknown,m,5Õ6Ó,3,Jan-98,yes
6,2,22,f,5Õ,III,Mar-99,Y
7,3,31,?,172,4,?,n
8,4,5,M,NaN,II,Last fall,N


### Fix Drug A and B
Drop Drug A column and crete two new column with binary classification if individuals have taken drug A or B (1) ot not (0).

In [154]:
drg.drop(["Drug A"], axis=1, inplace=True)
drg.dropna(how='all', inplace=True) # Drop all rows that only contain NaN
drg["Drug A"] = [1,1,1,0,0,0,0]
drg["Drug B"] = [0,0,0,1,1,1,1]
drg

,Age of patient,Patient gender,Height,Tumor stage,Date enrolled,Complications?,Drug A,Drug B
0,25,Male,61Ó,1,1/15/99,no,1,0
1,?,female,5Õ8Ó,2,2/1/99,y,1,0
2,40,male,120cm,IV,2/13/99,No,1,0
5,unknown,m,5Õ6Ó,3,Jan-98,yes,0,1
6,22,f,5Õ,III,Mar-99,Y,0,1
7,31,?,172,4,?,n,0,1
8,5,M,NaN,II,Last fall,N,0,1


### Transform Age of Patient to Numeric and impute missing values
Change the age of the patient column to a noumeric type. 
First Drop all the data_columns from the drg, and then use a join to put them back in after passing them through pd.to_numeric (with option 'coerce', such that all non-numeric entries are converted to NaN).
Follow up by imputing NaN ages with the average of all the other patients ages

In [155]:
drg =(drg.drop(["Age of patient"], axis=1)
         .join(drg["Age of patient"].apply(pd.to_numeric, errors='coerce'))) # convert to numeric
drg["Age of patient"] = drg["Age of patient"].fillna(round(drg["Age of patient"].mean()))

drg

,Patient gender,Height,Tumor stage,Date enrolled,Complications?,Drug A,Drug B,Age of patient
0,Male,61Ó,1,1/15/99,no,1,0,25.0
1,female,5Õ8Ó,2,2/1/99,y,1,0,25.0
2,male,120cm,IV,2/13/99,No,1,0,40.0
5,m,5Õ6Ó,3,Jan-98,yes,0,1,25.0
6,f,5Õ,III,Mar-99,Y,0,1,22.0
7,?,172,4,?,n,0,1,31.0
8,M,NaN,II,Last fall,N,0,1,5.0


### Transform Height Column into numeric height in cm
Transform Height Column into cm and format into a double. Inches (") are read in with Ó symbol (1cm = 2.54inches) and fee with Õ symbol (1 feet = 30.48 cm).

In [156]:
# replace NaN temporaryely with 0 so we can loop over them
drg["Height"].fillna(value='X', inplace = True)
newr =[]
tmprow = [] # Write in temp numbers for one row e.g. for inches and feet. 

for index, row in drg.iterrows():
    st = 0
    cnt = 0
    if row.Height.isdigit():
        newr.append(int(row.Height))
    else:
        for ltr in row.Height:
            if ltr.isdigit():
                cnt += 1
            else:
                if ltr == 'Ó':
                    tmprow.append(int((row.Height[st:st+cnt]))*2.54) 
                elif ltr == 'Õ':
                    tmprow.append(int((row.Height[st:st+cnt]))*30.48)
                elif ltr == 'c':
                    tmprow.append(int((row.Height[st:st+cnt])))
                elif ltr == 'X':
                    tmprow.append(0)
                else:
                    print("Warning!" , ltr)
                st = cnt + 1
                cnt = 0
        newr.append(round(sum(tmprow)))
        tmprow = []
newr

Warning! m


[155, 173, 120, 168, 152, 172, 0]

In [157]:
drg.Height = newr
drg['Height'].replace([0], np.NaN, inplace=True) # Reset the 0 height back to NaN
drg['Height'].fillna(round(drg["Height"].mean()),inplace=True) #impute NaN with average of all heights
drg

,Patient gender,Height,Tumor stage,Date enrolled,Complications?,Drug A,Drug B,Age of patient
0,Male,155.0,1,1/15/99,no,1,0,25.0
1,female,173.0,2,2/1/99,y,1,0,25.0
2,male,120.0,IV,2/13/99,No,1,0,40.0
5,m,168.0,3,Jan-98,yes,0,1,25.0
6,f,152.0,III,Mar-99,Y,0,1,22.0
7,?,172.0,4,?,n,0,1,31.0
8,M,157.0,II,Last fall,N,0,1,5.0
